# Digital Twin Data Acquisition

In this notebook, we will use Big Query and The Graph to obtain the signals required for the Rai Digital Twin. We will get state snapshots and calculate the absolute changes between time periods. 

### Signals  
* Price signals:
    * Eth price
    * Rai price in eth - need new source
    * Rai price in usd - need new source
    * Redemption price
    * Redemption rate.
* Uniswap data - https://thegraph.com/explorer/subgraph/uniswap/uniswap-v2
* Total debt = sum over SAFEs of debt
* Total collateral = sum over SAFEs of Collateral
* Total rai = directly observable from the RAI erc 20 contract (not sure how to get it from theGraph), further note that total rai = principle debt
* Excess Liability = Total debt - total rai
* ETH in Uniswap = should be queriable from thegraph data on uniswap (different source subgraph) --> future replace this with total value in ETH of assets RAI is paired with on Uniswap instances
* RAI in Uniswap = should be queriable from thegraph data on uniswap
* Debt ceiling = directly observable from RAI subgraph
* debt as fraction of debt ceiling = total debt/debt ceiling
* floating RAI = total rai - rai in uniswap
* floating RAI as fraction of total RAI = floating RAI /total RAI
another thing i am interested in from uniswap is the yield per unit liquidity
* total liquidity = price_usd_asset1 * quantity_of_asset1 + price_usd_asset2*quantity_of_asset2
* volume_per_period = take directly from contract
* fees_per_period = measure directly or compute from volumes
* revenues in token1 and token2
* value of profits in USD
* yields = profit in usd per total liquidity in USD of the whole pool
* Ratio of yields to total liquidity

In [1]:
# import libraries
import os
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import json
import requests
# The Graph URL
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'

%matplotlib inline

# constants
constant = 1000000000000000000

#defining creditionals
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/aclarkdata/Downloads/raidata-5e1723d42cf5.json'

# initializing agent
client = bigquery.Client()


In [16]:
# SQL query
sql = """
SELECT * 
FROM `blockchain-etl.ethereum_rai.ChainlinkMedianETHUSD_event_UpdateResult`
ORDER By block_timestamp DESC

"""

eth_price_next_OSM = client.query(sql).to_dataframe()
eth_price_next_OSM['ETH Price (Next OSM)'] = eth_price_next_OSM['medianPrice'].astype(float)/constant
# subset
eth_price_next_OSM = eth_price_next_OSM[['block_number','ETH Price (Next OSM)']]

eth_price_next_OSM.head()

,block_number,ETH Price (Next OSM)
0,12271233,2129.458042
1,12270946,2219.590000
2,12270590,2244.796301
3,12270259,2236.288682
4,12269966,2218.569789


In [18]:
# SQL query
sql = """
SELECT 
* 
FROM `blockchain-etl.ethereum_rai.OSM_event_UpdateResult`
ORDER By block_timestamp DESC

"""

eth_price_OSM = client.query(sql).to_dataframe()
eth_price_OSM['ETH Price (OSM)'] = eth_price_OSM['newMedian'].astype(float)/constant
# subset
eth_price_OSM = eth_price_OSM[['block_number','ETH Price (OSM)']]

eth_price_OSM.head()

,block_number,ETH Price (OSM)
0,12317786,2508.038362
1,12317484,2501.319518
2,12317232,2517.779640
3,12316966,2495.956312
4,12316689,2500.020000


## TODO: Can we find more data from the system state?

In [4]:
blocknumbers = eth_price_OSM.block_number.values.tolist()

state = []
for i in blocknumbers:
    query = '''
    {
      systemState(block: {number:%s},id:"current") { 
        coinUniswapPair {
          reserve0
          reserve1
        }
        currentCoinMedianizerUpdate{
          value
        }
        currentRedemptionRate {
          eightHourlyRate
          annualizedRate
          hourlyRate
          createdAt
        }
        currentRedemptionPrice {
          value
        }
        erc20CoinTotalSupply
        globalDebt
        globalDebtCeiling
        totalActiveSafeCount
        coinAddress
        wethAddress
        systemSurplus
        debtAvailableToSettle
        lastPeriodicUpdate
        createdAt
        createdAtBlock
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['systemState']
    state.append(s)
        
systemState = pd.DataFrame(state)

systemState['block_number'] = blocknumbers


systemState.head()


,coinAddress,coinUniswapPair,createdAt,createdAtBlock,currentCoinMedianizerUpdate,currentRedemptionPrice,currentRedemptionRate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,lastPeriodicUpdate,systemSurplus,totalActiveSafeCount,wethAddress,block_number
0,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '17955896.051624583496097305', 'r...",1529678381,5834717,{'value': '3.075324104055127061'},{'value': '3.017154246723011598365320866'},{'annualizedRate': '0.832398103667229502786879...,111425.8088961337724517833149917351,28744527.518921051043605948,29147704.72479329583837851734935911,115792089237316195423570985008687.9,1619457925,221365.5836013174610964248562207446,660,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12317484
1,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '17902491.808771992859330775', 'r...",1529678381,5834717,{'value': '3.075324104055127061'},{'value': '3.017186844181851205855771092'},{'annualizedRate': '0.816140306791344046568995...,111425.8088961337724517833149917351,28734527.518921051043605948,29137496.52044991614352952340749853,115792089237316195423570985008687.9,1619451339,221305.5631871845973015984898353958,660,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12317232
2,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '17962509.038372603200461905', 'r...",1529678381,5834717,{'value': '3.075324104055127061'},{'value': '3.017329328562231229633695135'},{'annualizedRate': '0.816140306791344046568995...,111425.8088961337724517833149917351,28734527.518921051043605948,29137496.52044991614352952340749853,115792089237316195423570985008687.9,1619451339,221305.5631871845973015984898353958,660,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12316966
3,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '17972329.889145352165250906', 'r...",1529678381,5834717,{'value': '3.081872665513482397'},{'value': '3.017329328562231229633695135'},{'annualizedRate': '0.816140306791344046568995...,111425.8088961337724517833149917351,28701909.018921051043605948,29104746.84754927119010607575099122,115792089237316195423570985008687.9,1619447867,221277.9080764961921013654700603952,660,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12316689
4,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '17975726.653321064193436411', 'r...",1529678381,5834717,{'value': '3.081872665513482397'},{'value': '3.017448200284182559470950147'},{'annualizedRate': '0.816140306791344046568995...,111425.8088961337724517833149917351,28650964.713483660930301976,29053388.86275158725423568656167433,115792089237316195423570985008687.9,1619434832,221173.7930529614396085773501051935,662,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12316405


In [5]:
systemState.tail()

,coinAddress,coinUniswapPair,createdAt,createdAtBlock,currentCoinMedianizerUpdate,currentRedemptionPrice,currentRedemptionRate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,lastPeriodicUpdate,systemSurplus,totalActiveSafeCount,wethAddress,block_number
1685,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",1529678381,5834717,{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,1613394096,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861502
1686,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",1529678381,5834717,None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,1613384616,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861237
1687,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",1529678381,5834717,None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,1613384616,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861008
1688,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",1529678381,5834717,None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.522357955470745187436691184972,115792089237316195423570985008687.9,1613338681,0.06853325908830422221834559248594176,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11860755
1689,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,None,1529678381,5834717,None,None,None,0,0,0,0,0,0,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11848351


In [27]:
systemState.coinUniswapPair.values[0]

{'reserve0': '17955896.051624583496097305',
 'reserve1': '21802.518885612138851202'}

In [26]:
systemState.currentCoinMedianizerUpdate.values[0]

{'value': '3.075324104055127061'}

In [6]:
systemState.drop(systemState.tail(4).index,
        inplace = True)

In [7]:
systemState['RedemptionRateAnnualizedRate'] = systemState.currentRedemptionRate.apply(lambda x: x['annualizedRate'])    
systemState['RedemptionRateHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['hourlyRate'])
systemState['RedemptionRateEightHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['eightHourlyRate'])
systemState['RedemptionPrice'] = systemState.currentRedemptionPrice.apply(lambda x: x['value'])
systemState['RAIInUniswapV2(RAI/ETH)'] = systemState.coinUniswapPair.apply(lambda x: x['reserve0'])
del systemState['currentRedemptionRate']
del systemState['currentRedemptionPrice']
systemState['RedemptionRateAnnualizedRate'] = systemState['RedemptionRateAnnualizedRate'].astype(float)
systemState['RedemptionRateHourlyRate'] = systemState['RedemptionRateHourlyRate'].astype(float)
systemState['RedemptionRateEightHourlyRate'] = systemState['RedemptionRateEightHourlyRate'].astype(float)
systemState['RedemptionPrice'] = systemState['RedemptionPrice'].astype(float)
systemState['RAIInUniswapV2(RAI/ETH)'] = systemState['RAIInUniswapV2(RAI/ETH)'].astype(float)


# subset
systemState = systemState[['debtAvailableToSettle','erc20CoinTotalSupply','globalDebt',
                           'globalDebtCeiling','systemSurplus','totalActiveSafeCount',
                           'block_number', 'RedemptionRateAnnualizedRate',
                           'RedemptionRateHourlyRate', 'RedemptionRateEightHourlyRate',
                        'RedemptionPrice', 'RAIInUniswapV2(RAI/ETH)']]

In [50]:
hourly = []
for i in blocknumbers[:-1]:
    query = '''
    {
      hourlyStats(block: {number:%s}) { 
        marketPriceUsd # price of COIN in USD (uni pool price * ETH median price)
        marketPriceEth # Price of COIN in ETH (uni pool price)
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['hourlyStats'][0]
    hourly.append(s)
        
hourlyStats = pd.DataFrame(hourly)

hourlyStats['block_number'] = blocknumbers[:-1]

hourlyStats.head()


,marketPriceEth,marketPriceUsd,block_number
0,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,12317484
1,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,12317232
2,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,12316966
3,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,12316689
4,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,12316405


In [11]:
safehistories = []
for i in blocknumbers[:-1]:
    query = '''
    {
      safes(block: {number:%s}) {
            collateral
            debt
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['safes']
    t = pd.DataFrame(s)
    t['collateral'] = t['collateral'].astype(float)
    t['debt'] = t['debt'].astype(float)
    safehistories.append(pd.DataFrame(t.sum().to_dict(),index=[0]))

safe_history = pd.concat(safehistories)


In [33]:
safe_history.head()

,collateral,debt
0,1545.8032,336053.912094
0,1545.8032,336053.912094
0,1545.8032,336053.912094
0,1545.8032,336053.912094
0,1545.8032,336053.912094


In [43]:
safe_history['block_number'] = blocknumbers[:-1]
safe_history.reset_index(inplace=True)
del safe_history['index']

# safe_history['collateral_diff'] = safe_history['collateral'].diff()
# safe_history['debt_diff'] = safe_history['debt'].diff()
# safe_history.head()

## TODO: Combine, delta everything. See what we are missing

In [51]:
states = hourlyStats.merge(systemState,how='inner',on='block_number')
states = states.merge(safe_history,how='inner',on='block_number')

In [52]:
prices = eth_price_OSM.merge(eth_price_next_OSM,how='inner',on='block_number')
prices.head()

,block_number,ETH Price (OSM),ETH Price (Next OSM)
0,12182570,2117.650000,2104.170974
1,12162180,2052.464741,2072.713775
2,12158673,1991.766574,1998.000000
3,12134065,1767.251649,1777.289555
4,12110194,1602.851185,1607.400000


In [53]:
states = states.merge(prices,how='inner',on='block_number')

In [54]:
states.head()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RAIInUniswapV2(RAI/ETH),collateral,debt,ETH Price (OSM),ETH Price (Next OSM)
0,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,12182570,0,35744128.158530230379891959,35983694.85927260111721464936215804,115792089237316195423570985008687.9,187338.5959485606686820321694960594,984,0.893578,0.999987,0.999897,3.030509,3.104180e+07,6616.2132,1.363851e+06,2117.650000,2104.170974
1,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,12162180,13643.85936176352503894904953636526,36621857.506916628047720509,36868980.12807649336556033467084887,115792089237316195423570985008687.9,184264.3263143241582880245129434617,981,0.846295,0.999981,0.999848,3.033423,3.203570e+07,6921.6132,1.398615e+06,2052.464741,2072.713775
2,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,12158673,13643.85936176352503894904953636526,35929070.304637658517611759,36175097.84301315391564853414540481,115792089237316195423570985008687.9,183712.5657625425875127865653801174,983,0.911912,0.999989,0.999916,3.034000,3.150928e+07,6921.6132,1.398615e+06,1991.766574,1998.000000
3,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,12134065,24653.20536874873589640898769725803,41104313.60308670894550387,41353159.82965628864125731758421892,115792089237316195423570985008687.9,179640.3035482553016037784209888087,999,1.242260,1.000025,1.000198,3.033300,3.622565e+07,7019.6132,1.418540e+06,1767.251649,1777.289555
4,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,12110194,49755.45276765162608995551015657265,40925246.895913380004668756,41191190.06752416993112726675898989,115792089237316195423570985008687.9,175617.172673347380380196043242654,993,1.395665,1.000038,1.000304,3.027886,3.442781e+07,6495.1132,1.338084e+06,1602.851185,1607.400000


In [56]:
states.tail()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RAIInUniswapV2(RAI/ETH),collateral,debt,ETH Price (OSM),ETH Price (Next OSM)
14,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,11874799,846.1745167717419600463715557405962,421343.030490994254846172,422302.408088138116419818529792371,115792089237316195423570985008687.9,106.0103992741321969166359161218292,30,0.999887,1.0,1.0,3.139998,262204.136069,2187.79736,421366.076801,1833.010000,1819.870854
15,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,11870974,846.1745167717419600463715557405962,9784.515018562104188503,10733.07228393042461568621532158055,115792089237316195423570985008687.9,101.9206100615458255061762292734445,10,0.999666,1.0,1.0,3.139999,1153.422981,53.17336,9885.111302,1768.260000,1782.580000
16,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,11870443,846.1745167717419600463715557405962,9054.515018562104188503,10002.82076364474431855732514988873,115792089237316195423570985008687.9,101.8737677218308615894326598409466,10,0.999710,1.0,1.0,3.139999,998.881222,52.29336,9155.243869,1747.760000,1753.894121
17,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,11865824,846.1745167717419600463715557405962,7056.799518562104188503,8004.909406251387781073606551202994,115792089237316195423570985008687.9,101.7451032740474329040709042772893,7,1.000000,1.0,1.0,3.140000,1053.964708,33.99236,7157.848499,1816.150000,1813.156680
18,0.002589462210048543650949087456484599,4.709364363675082915930020079963684,11861791,0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,1.000000,1.0,1.0,3.140000,801.933727,16.89236,3048.358490,1805.792735,1803.891149


In [58]:
states = states.astype(float)
states.diff()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RAIInUniswapV2(RAI/ETH),collateral,debt,ETH Price (OSM),ETH Price (Next OSM)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,-20390.0,13643.859362,8.777293e+05,8.852853e+05,0.0,-3074.269634,-3.0,-0.047283,-6.206012e-06,-4.964255e-05,2.913399e-03,9.938989e+05,305.40000,3.476404e+04,-65.185259,-31.457199
2,0.0,0.0,-3507.0,0.000000,-6.927872e+05,-6.938823e+05,0.0,-551.760552,2.0,0.065617,8.524507e-06,6.818899e-05,5.775022e-04,-5.264169e+05,0.00000,0.000000e+00,-60.698166,-74.713775
3,0.0,0.0,-24608.0,11009.346007,5.175243e+06,5.178062e+06,0.0,-4072.262214,16.0,0.330348,3.529062e-05,2.823391e-04,-7.000654e-04,4.716371e+06,98.00000,1.992516e+04,-224.514925,-220.710445
4,0.0,0.0,-23871.0,25102.247399,-1.790667e+05,-1.619698e+05,0.0,-4023.130875,-6.0,0.153405,1.329249e-05,1.063633e-04,-5.414652e-03,-1.797841e+06,-524.50000,-8.045681e+04,-164.400464,-169.889555
5,0.0,0.0,-20295.0,-10003.778410,2.478484e+06,2.459652e+06,0.0,-5310.829444,28.0,-0.182864,-1.603225e-05,-1.282850e-04,-7.940291e-03,4.755973e+06,1693.60000,5.459812e+05,178.718815,164.070000
6,0.0,0.0,-99166.0,829315.906668,-1.071340e+07,-9.927190e+06,0.0,-24301.977425,-182.0,-0.912071,-1.591774e-04,-1.272906e-03,1.026785e-02,-1.029814e+07,-1810.13000,-7.453038e+05,-95.464727,-90.775771
7,0.0,0.0,-46294.0,-564582.688615,3.739289e+06,3.071334e+06,0.0,-96800.673657,-37.0,-0.019669,-7.720535e-06,-6.170334e-05,6.588603e-02,2.647082e+06,-582.42000,-2.013835e+04,-297.015654,-345.262070
8,0.0,0.0,-24981.0,-58785.451749,-9.895652e+06,-9.968018e+06,0.0,-11871.686602,-33.0,-0.005846,-2.399183e-06,-1.917385e-05,2.047325e-02,-1.093921e+07,2896.63780,7.744773e+05,295.445521,350.982028
9,0.0,0.0,-13527.0,-228278.034320,-1.341254e+05,-3.960962e+05,0.0,-33346.016259,66.0,0.564621,1.273471e-04,1.018181e-03,9.519656e-03,-8.041164e+03,1413.20000,5.308804e+05,189.907214,178.484206
